# Assignment02: Find Academic Keyword List
Academic Keywords are the words we seldom use ordinarily, but often use in Academic articles. "This shows" and "in conclusion" are examples of Academic Keywords. This assignment want you to use Rank Ratio and compare two dataset to find Academic Keyword List(AKL).
<br><br>
One dataset is taken from [`British Academic Written English Corpus(BAWE)`](https://www.coventry.ac.uk/research/research-directories/current-projects/2015/british-academic-written-english-corpus-bawe/), which collect a record of proficient university-level student writing. Hence, BAWE can be seen as Academic data. Another one is called [`Web1T`](https://catalog.ldc.upenn.edu/LDC2006T13), which is presented by Google. Web1T colloct 1 trillion words of English Web, so we can treat it as the representative of common words. 

## N-gram counting
This part is almost same as what you need to do in Assignment01. The way to find N-gram is the same as Assignment01. However, tokenization and calculating frequency are a little different. The rules of tokenization in this Assignment are:
 1. Ignore case (e.g., "The" is the same as "the")
 2. Split by white spaces <s>and punctuations</s>
 3. Ignore all punctuation
<br><br>

As for calculating frequency, we want you calculating <u>document frequency</u> in this Assignment. <br>What is document frequency? 
<br>Article 1: 
> We all know that water masses in the ocean are thought to be transferred by the wind. ...

Althought there are at least 2 "the" in Article 1, the document frequency of "the" is still 1 in this article.<br> No mater how many times does "the" show up in Article 1, the document frquency of it is always 1.<br>
Article 2: 
> The film Dantes Peak is about a dormant volcano that suddenly erupts and threatens the nearby town. ...

Considering the Article 1 and 2, the document frequency of "the" is 2 now.<br>
Document frequency can reduce the influence of terms, like "NLP".
<br><br>
<span style="color: red">[ TODO ]</span> Try to modify the functions coded in Assignment01 to <u>calculate document frequencies of all unigram.</u>.

Google has calculated the frequency of N-gram, so you only need to do it on BAWE.

In [ ]:
'''
I do the assignments on Google Colab, so this section just helps me mounting
Google Drive to Colab directories.
'''
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab\ Notebooks/week02/

In [ ]:
def tokenize(text):
    import re

    # [ TODO ] transform to lower case
    text = text.lower()
    
    # ignore punctuations
    '''
    Originally, I deleted all punctuations from:
    -->  ",.'[]\"/-():;%=<>?!*~|&+{_}\@$#`^"  <--

    The rank ratio seems not very similar to TA's sample.
    So I tried a lot of combinations of punctuations and finally got a quite
    good one:
    --> "=<>?!*~|&+{_}\@$#`^" <--
    (What I mean good is about the rank ratio, I think it should be similar to
    the sample).
    '''
    puncs = "=<>?!*~|&+{_}\@$#`^"
    for punc in puncs:
        text = text.replace(punc, '')
    
    # [ TODO ] seperate the words
    tokens = re.split('\s+', text)
    '''
    In assignment 1, I use \W+ to separate words by any non-English and
    non-numeric charaters.
    But now we only split tokens by spaces, so I change it to \s+ which contains
    spaces, tabs, newlines, etc.
    '''
    return tokens

In [ ]:
def calculate_frequency(tokens):
    # [ TODO ]
    # frequency = {...}
    
    frequency = {token: 1 for token in set(tokens)}
    '''
    Here we care about DOCUMENT FREQUENCY, meaning that words appearing more
    than once can only be counted as one (in a single document).
    '''
    return frequency

In [ ]:
def get_ngram(tokens, n=2):
    # [ TODO ]
    ngram_tokens = []
    for i in range(len(tokens) - n):
        ngram_tokens.append(" ".join(tokens[i:i + n]))
    return ngram_tokens

In [ ]:
import os

In [ ]:
file_path = os.path.join('data', 'BAWE.txt')
BAWE_unigram = []
#### [ TODO ] calculate document frequency of unigram in BAWE
import collections
BAWE_unigram_set = set()
BAWE_unigram_frequency = collections.defaultdict(int)
'''
I use BAWE_unigram_set to collect all unique unigrams, and use
BAWE_unigram_frequency to accumulate each token with its document frequencies.
'''
with open(file_path, 'r') as f:
    for text in f.readlines():
        tokens = tokenize(text)
        BAWE_unigram_set = BAWE_unigram_set.union(set(tokens))
        doc_frequency = calculate_frequency(tokens)
        for k, v in doc_frequency.items():
            BAWE_unigram_frequency[k] += v
BAWE_unigram = list(BAWE_unigram_set)

In [ ]:
# Read Web1T Data
file_path = os.path.join('data', 'Web1T_unigram.txt')
Web1T_unigram_counter = {}
with open(file_path,'r') as f:
    for line in f.readlines():
        line=line.rstrip("\n").split("\t")
        Web1T_unigram_counter[line[0]] = int(line[1])

## Rank
Rank unigrms by their frequencies. The higher the frequency, the higher the rank.(The most frequent unigram ranks 1.)<br>
<span style="color: red">[ TODO ]</span> <u>Rank unigrams for Web1T and BAWE.</u>.

In [ ]:
Web1T_unigram_Rank = {}
#### [ TODO ] Rank unigrams for Web1T
'''
I found out that if I give the ranks one by one (e.g. 1, 2, 3, ...) without
caring unigrams with duplicate frequencies, the rank ratios are not accurate.
Only when I take account of duplicate frequencies, meaning that same frequencies
should have same ranks, the rank ratios just become more accurate a bit.
'''
prev_freq = -1
rank = 0
acc = 1
for k, v in sorted(Web1T_unigram_counter.items(), key=lambda elem: elem[1], reverse=True):
    if prev_freq != v:
        rank += acc
        acc = 1
    else:
        acc += 1
    Web1T_unigram_Rank[k] = rank
    prev_freq = v

In [ ]:
BAWE_unigram_Rank = {}
#### [ TODO ] Rank unigrams for BAWE
prev_freq = -1
rank = 0
acc = 1
for k, v in sorted(BAWE_unigram_frequency.items(), key=lambda elem: elem[1], reverse=True):
    if prev_freq != v:
        rank += acc
        acc = 1
    else:
        acc += 1
    BAWE_unigram_Rank[k] = rank
    prev_freq = v

## Calculate Rank Ratio
In this step, you need to map the same unigram in two dataset, and caalculate the Rank Ratio of unigram in BAWE.  <br>Please follow the formula for calculating Rank Ratio:<br> 
<br>
<img src="https://imgur.com/vmK7Q1K.jpg" width=30%><br>
If the unigram doesn't appear in Web1T, the rank of it is treated as 1.

<span style="color: red">[ TODO ]</span> Please calculate all rank ratios of unigrams in BAWE.

In [ ]:
#### [TODO] calculate all rank ratios of unigrams in BAWE
BAWE_unigram_rank_ratio = {}
for unigram in BAWE_unigram:
    BAWE_unigram_rank_ratio[unigram] = Web1T_unigram_Rank.get(unigram, 1) / BAWE_unigram_Rank[unigram]

## sort the result
<span style="color: red">[ TODO ]</span> Please show top 30 uigrams in Rank Ratio and the value of their Rank Ratio in this format: 
<br>
<img src="https://imgur.com/AEkiCRr.jpg" width=50%>

In [ ]:
#### [ TODO ] show the result
print("rank\tunigram\t\t\tRank Ratio")
for i, (k, v) in enumerate(sorted(BAWE_unigram_rank_ratio.items(), key=lambda elem: elem[1], reverse=True)[:30], start=1):
    print("{}{}{}".format(str(i).ljust(8, ' '), str(k).ljust(24, ' '), str(round(v, 4)).ljust(16, ' ')))

rank	unigram			Rank Ratio
1       cannot                  666.2188  
2       conclusion              19.0425   
3       whilst                  17.126    
4       emphasises              16.6271   
5       trudgill                16.06     
6       sibilance               15.9606   
7       suggests                15.9207   
8       argued                  15.5319   
9       generalisability        15.4392   
10      essay                   14.7581   
11      legitimising            14.6206   
12      argues                  14.3983   
13      therefore               14.163    
14      plosives                14.0546   
15      analysing               13.676    
16      foregrounded            13.4585   
17      firstly                 13.2532   
18      hypothesise             13.1172   
19      emphasising             13.0346   
20      analyse                 12.918    
21      stoppered               12.9052   
22      analysed                12.8473   
23      hypothesised        

## for Bigrams
<span style="color: red">[ TODO ]</span> Do the Same Thing for Bigrams

In [ ]:
import re

def tokenize(text):
    # transform to lower case
    text = text.lower()
    
    # ignore punctuations
    puncs = ",.'[]\"/-():;%=?!*~|&+{_}\@$#`^"
    for punc in puncs:
        text = text.replace(punc, '')
    
    # seperate the words
    tokens = re.split('\s+', text)
    return tokens

In [ ]:
file_path = os.path.join('data', 'BAWE.txt')
BAWE_bigram = []
# calculate document frequency of bigram in BAWE
import collections
BAWE_bigram_set = set()
BAWE_bigram_frequency = collections.defaultdict(int)
with open(file_path, 'r') as f:
    for text in f.readlines():
        tokens = tokenize(text)
        bigram = get_ngram(tokens, n=2)  # get bigrams
        BAWE_bigram_set = BAWE_bigram_set.union(set(bigram))
        doc_frequency = calculate_frequency(bigram)
        for k, v in doc_frequency.items():
            BAWE_bigram_frequency[k] += v
BAWE_bigram = list(BAWE_bigram_set)

In [ ]:
# read Web1T bigram data
file_path = os.path.join('data', 'Web1T_bigram.txt')
Web1T_bigram_counter = {}
with open(file_path,'r') as f:
    for line in f.readlines():
        line = line.rstrip("\n").split("\t")
        Web1T_bigram_counter[line[0]] = int(line[1])

In [ ]:
# calculate rank of bigram in Web1T
Web1T_bigram_Rank = {}
prev_freq = -1
rank = 0
acc = 1
for k, v in sorted(Web1T_bigram_counter.items(), key=lambda elem: elem[1], reverse=True):
    if prev_freq != v:
        rank += acc
        acc = 1
    else:
        acc += 1
    Web1T_bigram_Rank[k] = rank
    prev_freq = v

In [ ]:
## calculate rank of bigram in BAWE
BAWE_bigram_Rank = {}
prev_freq = -1
rank = 0
acc = 1
for k, v in sorted(BAWE_bigram_frequency.items(), key=lambda elem: elem[1], reverse=True):
    if prev_freq != v:
        rank += acc
        acc = 1
    else:
        acc += 1
    BAWE_bigram_Rank[k] = rank
    prev_freq = v

In [ ]:
# calculate rank ratio of bigram in BAWE
BAWE_bigram_rank_ratio = {}
for bigram in BAWE_bigram:
    BAWE_bigram_rank_ratio[bigram] = Web1T_bigram_Rank.get(bigram, 1) / BAWE_bigram_Rank[bigram]

In [ ]:
# print results
print("rank\tbigram\t\t\tRank Ratio")
for i, (k, v) in enumerate(sorted(BAWE_bigram_rank_ratio.items(), key=lambda elem: elem[1], reverse=True)[:30], start=1):
    print("{}{}{}".format(str(i).ljust(8, ' '), str(k).ljust(24, ' '), str(round(v, 4)).ljust(16, ' ')))

rank	bigram			Rank Ratio
1       in conclusion           517.3043  
2       however this            418.7165  
3       however the             377.0602  
4       however in              285.8129  
5       however it              247.1333  
6       this essay              228.844   
7       however there           218.7548  
8       the british             186.9405  
9       the european            155.2256  
10      this suggests           139.9115  
11      this shows              107.3271  
12      analysis the            93.0619   
13      therefore it            91.062    
14      however a               90.7214   
15      see appendix            90.5267   
16      therefore the           87.2732   
17      method the              77.1569   
18      conclusion in           65.8483   
19      however he              63.551    
20      the uk                  63.2821   
21      however to              62.0589   
22      the united              61.6882   
23      a persons            

### Optional Problem:

In [ ]:
# read all AKL words
AKL = "ability, absence, account, achievement, act, action, activity, addition, adoption, adult, advance, advantage, advice, age, aim, alternative, amount, analogy, analysis, application, approach, argument, aspect, assertion, assessment, assistance, association, assumption, attempt, attention, attitude, author, awareness, balance, basis, behaviour, being, belief, benefit, bias, birth, capacity, case, category, cause, centre, challenge, change, character, characteristic, choice, circumstance, class, classification, code, colleague, combination, commitment, committee, communication, community, comparison, complexity, compromise, concentration, concept, conception, concern, conclusion, condition, conduct, conflict, consensus, consequence, consideration, constraint, construction, content, contradiction, contrast, contribution, control, convention, correlation, country, creation, crisis, criterion, criticism, culture, damage, data, debate, decision, decline, defence, definition, degree, demand, description, destruction, determination, development, difference, difficulty, dilemma, dimension, disadvantage, discovery, discrimination, discussion, distinction, diversity, division, doctrine, effect, effectiveness, element, emphasis, environment, error, essence, establishment, evaluation, event, evidence, evolution, examination, example, exception, exclusion, existence, expansion, experience, experiment, explanation, exposure, extent, extreme, fact, factor, failure, feature, female, figure, finding, force, form, formation, function, future, gain, group, growth, guidance, guideline, hypothesis, idea, identity, impact, implication, importance, improvement, increase, indication, individual, influence, information, insight, instance, institution, integration, interaction, interest, interpretation, intervention, introduction, investigation, isolation, issue, kind, knowledge, lack, learning, level, likelihood, limit, limitation, link, list, literature, logic, loss, maintenance, majority, male, manipulation, mankind, material, means, measure, medium, member, method, minority, mode, model, motivation, movement, need, network, norm, notion, number, observation, observer, occurrence, operation, opportunity, option, organisation, outcome, output, parallel, parent, part, participant, past, pattern, percentage, perception, period, person, personality, perspective, phenomenon, point, policy, population, position, possibility, potential, practice, presence, pressure, problem, procedure, process, production, programme, progress, property, proportion, proposition, protection, provision, publication, purpose, quality, question, range, rate, reader, reality, reason, reasoning, recognition, reduction, reference, relation, relationship, relevance, report, representative, reproduction, requirement, research, resistance, resolution, resource, respect, restriction, result, review, rise, risk, role, rule, sample, scale, scheme, scope, search, section, selection, sense, separation, series, service, set, sex, shift, significance, similarity, situation, skill, society, solution, source, space, spread, standard, statistics, stimulus, strategy, stress, structure, subject, success, summary, support, survey, system, target, task, team, technique, tendency, tension, term, theme, theory, tolerance, topic, tradition, transition, trend, type, uncertainty, understanding, unit, use, validity, value, variation, variety, version, view, viewpoint, volume, whole, work, world, accept, account (for), achieve, acquire, act, adapt, adopt, advance, advocate, affect, aid, aim, allocate, allow, alter, analyse, appear, apply, argue, arise, assert, assess, assign, associate, assist, assume, attain, attempt, attend, attribute, avoid, base, be, become, benefit, can, cause, characterise, choose, cite, claim, clarify, classify, coincide, combine, compare, compete, comprise, concentrate, concern, conclude, conduct, confine, conform, connect, consider, consist, constitute, construct, contain, contrast, contribute, control, convert, correspond, create, damage, deal, decline, define, demonstrate, depend, derive, describe, design, destroy, determine, develop, differ, differentiate, diminish, direct, discuss, display, distinguish, divide, dominate, effect, eliminate, emerge, emphasize, employ, enable, encounter, encourage, enhance, ensure, establish, evaluate, evolve, examine, exceed, exclude, exemplify, exist, expand, experience, explain, expose, express, extend, facilitate, fail, favour, finance, focus, follow, form, formulate, function, gain, generate, govern, highlight, identify, illustrate, imply, impose, improve, include, incorporate, increase, indicate, induce, influence, initiate, integrate, interpret, introduce, investigate, involve, isolate, label, lack, lead, limit, link, locate, maintain, may, measure, neglect, note, obtain, occur, operate, outline, overcome, participate, perceive, perform, permit, pose, possess, precede, predict, present, preserve, prevent, produce, promote, propose, prove, provide, publish, pursue, quote, receive, record, reduce, refer, reflect, regard, regulate, reinforce, reject, relate, rely, remain, remove, render, replace, report, represent, reproduce, require, resolve, respond, restrict, result, retain, reveal, seek, select, separate, should, show, solve, specify, state, stimulate, strengthen, stress, study, submit, suffer, suggest, summarise, supply, support, sustain, tackle, tend, term, transform, treat, undermine, undertake, use, vary, view, write, yield, absolute, abstract, acceptable, accessible, active, actual, acute, additional, adequate, alternative, apparent, applicable, appropriate, arbitrary, available, average, basic, central, certain, clear, common, competitive, complete, complex, comprehensive, considerable, consistent, conventional, correct, critical, crucial, dependent, detailed, different, difficult, distinct, dominant, early, effective, equal, equivalent, essential, evident, excessive, experimental, explicit, extensive, extreme, far, favourable, final, fixed, following, formal, frequent, fundamental, future, general, great, high, human, ideal, identical, immediate, important, inadequate, incomplete, independent, indirect, individual, inferior, influential, inherent, initial, interesting, internal, large, late, leading, likely, limited, local, logical, main, major, male, maximum, mental, minimal, minor, misleading, modern, mutual, natural, necessary, negative, new, normal, obvious, original, other, overall, parallel, partial, particular, passive, past, permanent, physical, positive, possible, potential, practical, present, previous, primary, prime, principal, productive, profound, progressive, prominent, psychological, radical, random, rapid, rational, real, realistic, recent, related, relative, relevant, representative, responsible, restricted, scientific, secondary, selective, separate, severe, sexual, significant, similar, simple, single, so-called, social, special, specific, stable, standard, strict, subsequent, substantial, successful, successive, sufficient, suitable, surprising, symbolic, systematic, theoretical, total, traditional, true, typical, unique, unlike, unlikely, unsuccessful, useful, valid, valuable, varied, various, visual, vital, wide, widespread, above, accordingly, accurately, adequately, also, approximately, at best, basically, clearly, closely, commonly, consequently, considerably, conversely, correctly, directly, effectively, e.g., either, equally, especially, essentially, explicitly, extremely, fairly, far, for example, for instance, frequently, fully, further, generally, greatly, hence, highly, however, increasingly, indeed, independently, indirectly, inevitably, initially, in general, in particular, largely, less, mainly, more, moreover, most, namely, necessarily, normally, notably, often, only, originally, over, partially, particularly, potentially, previously, primarily, purely, readily, recently, relatively, secondly, significantly, similarly, simply, socially, solely somewhat, specifically, strongly, subsequently, successfully, thereby, therefore, thus, traditionally, typically, ultimately, virtually, wholly, widely, according to, although, an, as, as opposed to, as to, as well as, because, because of, between, both, by, contrary to, depending on, despite, due to, during, each, even though, fewer, first, former, from, for, given that, in, in addition to, in common with, in favour of, in relation to, in response to, in terms of, in that, in the light of, including, its, itself, latter, less, little, many, most, of, or, other than, per, prior to, provided, rather than, same, second, several, since, some, subject to, such, such as, than, that, the, their, themselves, these, third, this, those, to, unlike, upon, versus, whereas, whether, whether or not, which, within".split(", ")
print(AKL[:10])

['ability', 'absence', 'account', 'achievement', 'act', 'action', 'activity', 'addition', 'adoption', 'adult']


In [ ]:
def tokenize(text):
    import re

    # transform to lower case
    text = text.lower()
    
    # ignore punctuations
    puncs = ",.'[]\"/-():;%=<>?!*~|&+{_}\@$#`^"
    for punc in puncs:
        text = text.replace(punc, '')
    
    # seperate the words
    tokens = re.split('\s+', text)
    
    return tokens

def get_ngram(tokens, n=2):
    ngram_tokens = []
    for i in range(len(tokens) - n):
        ngram_tokens.append(" ".join(tokens[i:i + n]))
    return ngram_tokens

def calculate_frequency(tokens):
    import collections

    frequency = collections.Counter(tokens)
    return frequency

def calculate_rank(freqs):
    ranks = {}
    prev_freq = -1
    rank = 0
    acc = 1
    for k, v in sorted(freqs.items(), key=lambda elem: elem[1], reverse=True):
        if prev_freq != v:
            rank += acc
            acc = 1
        else:
            acc += 1
        ranks[k] = rank
        prev_freq = v
    return ranks

def calculate_rank_ratio(ranks1, ranks2, ngrams):
    # rank ratio = ranks1[ngram] / ranks2[ngram]
    rank_ratios = {}
    for ngram in ngrams:
        rank_ratios[ngram] = ranks1.get(ngram, 1) / ranks2[ngram]
    return rank_ratios


s = "Question Answering (QA) has been an area of active research. \
Recently, the state-of-the-art in QA research has been \
represented in the Text Retrieval Evaluation Conference (TREC) \
question answering track evaluations [Voorhees 2001], which \
involve retrieving a short (50 bytes long) answer to a set of test \
questions. In our work we address an aspect of question \
answering that was not a direct focus of the TREC QA track. We \
also consider a more general class of questions, where the \
answers may not be short, precise facts, and the user might be \
interested in multiple answers (e.g., consider the question “What \
are ways people can be motivated?”)."


### Unigram 
unigram_tokens = tokenize(s)
unigrams = set(unigram_tokens)
unigram_freqs = calculate_frequency(unigram_tokens)
unigram_ranks = calculate_rank(unigram_freqs)
unigram_rank_ratios = calculate_rank_ratio(Web1T_unigram_Rank, unigram_ranks, unigrams)

print("Unigram\nrank\tunigram\t\t\tRank Ratio\tIs In AKL")
target_words = ['question', 'answer', 'area', 'active', 'research']
for i, (k, v) in enumerate(sorted(unigram_rank_ratios.items(), key=lambda elem: elem[1], reverse=True), start=1):
    if k in target_words:
        print("{}{}{}{}".format(str(i).ljust(8, ' '), str(k).ljust(24, ' '), str(round(v, 4)).ljust(16, ' '), str('YES' if k in AKL else 'NO')))

### Bigram 109061539
bigram_tokens = get_ngram(unigram_tokens, n=2)
bigrams = set(bigram_tokens)
bigram_freqs = calculate_frequency(bigram_tokens)
bigram_ranks = calculate_rank(bigram_freqs)
bigram_rank_ratios = calculate_rank_ratio(Web1T_bigram_Rank, bigram_ranks, bigrams)
print("\n\nBigram\nrank\tbigram\t\t\tRank Ratio")
for i, (k, v) in enumerate(sorted(bigram_rank_ratios.items(), key=lambda elem: elem[1], reverse=True)[:30], start=1):
    print("{}{}{}".format(str(i).ljust(8, ' '), str(k).ljust(24, ' '), str(round(v, 4)).ljust(16, ' ')))

Unigram
rank	unigram			Rank Ratio	Is In AKL
12      question                197.3333        YES
24      answer                  59.5714         NO
28      active                  45.4286         YES
33      research                21.3333         YES
37      area                    12.0476         NO


Bigram
rank	bigram			Rank Ratio
1       active research         90625.6667
2       address an              85560.6667
3       test questions          54331.6667
4       we address              43644.6667
5       of question             35411.0   
6       recently the            34935.0   
7       which involve           34121.0   
8       are ways                27116.3333
9       an aspect               24734.3333
10      be short                24589.6667
11      work we                 16651.0   
12      also consider           14868.0   
13      of test                 9011.0    
14      research has            7112.6667 
15      questions in            6733.0    
16      in multiple

## TA's Notes

If you complete the Assignment, please use [this link](https://docs.google.com/spreadsheets/d/1QGeYl5dsD9sFO9SYg4DIKk-xr-yGjRDOOLKZqCLDv2E/edit#gid=40492256) to reserve demo time.  
The score is only given after TAs review your implementation, so <u>**make sure you make a appointment with a TA before you miss the deadline**</u> .  <br>After demo, please upload your assignment to eeclass. You just need to hand in this ipynb file and rename it as XXXXXXXXX(Your student ID).ipynb.
<br>Note that **late submission will not be allowed**.  